El objetivo de este notebook es mostrar como podemos aplicar una transformación a un QNodo. Para ello crearemos un circuito inicial y posteriormente veremos como aplicar una transformación para sustituir las puertas X por la concatenación de una puerta Y y una puerta Z.

In [1]:
import pennylane as qml

dev = qml.device("default.qubit", wires = 3)

@qml.qnode(dev)
def circuit():
    qml.PauliX(wires = 0)
    qml.SWAP(wires = [0,2])
    qml.Hadamard(wires = 1)
    qml.PauliX(wires = 1)
    qml.CRZ(2, wires = [1,2])
    return qml.state()
    
circuit()

print(qml.draw(circuit)())

 0: ──X─────╭SWAP──────────╭┤ State 
 1: ──H──X──│──────╭C──────├┤ State 
 2: ────────╰SWAP──╰RZ(2)──╰┤ State 



La transformación recorrerá las operaciones de una lista y hará la sustitución correspondiente.

In [2]:
@qml.qfunc_transform
def remove_X(tape):
    
    for op in tape.operations:
        if op.name != "PauliX":
            qml.apply(op)
            
        else:
            qml.PauliY(wires = op.wires)
            qml.PauliZ(wires = op.wires)
            
        
    for ms in tape.measurements:
        qml.apply(ms)

Finalmente, utilizando el propio nombre de la función como decorador aplicaremos la función al qnodo en cuestión.

In [3]:
import pennylane as qml

dev = qml.device("default.qubit", wires = 3)

@qml.qnode(dev)
@remove_X
def circuit():
    qml.PauliX(wires = 0)
    qml.SWAP(wires = [0,2])
    qml.Hadamard(wires = 1)
    qml.PauliX(wires = 1)
    qml.CRZ(2, wires = [1,2])
    return qml.state()
    
circuit()

print(qml.draw(circuit)())

 0: ──Y──Z─────╭SWAP──────────╭┤ State 
 1: ──H──Y──Z──│──────╭C──────├┤ State 
 2: ───────────╰SWAP──╰RZ(2)──╰┤ State 



Como podemos comprobar las puertas X han sido remplazadas correctamente.